In [1]:
import torch
import pgb
from example_modules import GPT2

mod = GPT2(nlayers=8,dropout=0)
context1 = torch.tensor([[ 464, 5440, 4534]])
d = {"src":context1}

In [2]:
import warnings ; warnings.filterwarnings("ignore")
GPT2_ag = pgb.make_all_graphs(mod,d,check_device_is_gpu=False)

In [3]:
list_kg = GPT2_ag.K_graph_list
list_kg_eco = pgb.graph_translator.S_list_to_K_list_eco(GPT2_ag.S_graph_list,mod,print_cc=True)

Connexe component n°0: [0]
Connexe component n°1: [1, 2, 3, 4, 5, 6, 7]
Connexe component n°2: [8]
Connexe component n°3: [9]
We now have only 4 blocks to handle, instead of 10


In [4]:
pgb.print_graph_list(list_kg_eco,name="list_kg_eco",open=False)

10 K_graphs in seq, with :
8+25+25+25+25+25+25+25+25+3 = 211 Comp nodes
7+28+28+28+28+28+28+28+29+3 = 235 Data nodes
=> total of 446 nodes


## Check the equivalence of list_kg and list_kg_eco

In [5]:
for i,(kg_real,kg_eco) in enumerate(zip(list_kg,list_kg_eco)):
    print(i,end=" ")
    kg_real.__eq__(kg_eco,force_order=True,raise_exception=True)
    # set.__iter__'s inner order depends on the order in which elmts were added.
    # Moreover, between a copy of a graph and a complete construction this 
    # order of insertion is obviously different. That's why I need to "force_order"
    # when checking .deps/.users equalities for each node.

0 1 2 3 4 5 6 7 8 9 

## Check perfect determinism of both functions

In [6]:
GPT2_ag2 = pgb.make_all_graphs(mod,d,check_device_is_gpu=False,bool_kg=False)
list_kg_eco2 = pgb.graph_translator.S_list_to_K_list_eco(GPT2_ag2.S_graph_list,mod)

# Here we don't even need to "force_order"
print(f"classic construction determinism : {GPT2_ag2.K_graph_list == list_kg}")
print(f"economic construction determinism : {list_kg_eco == list_kg_eco2}")

classic construction determinism : True
economic construction determinism : True


## Compare the time taken by classic/economic methodes.

In [7]:
import time

n_layers = 8
n_test = 3
mod = GPT2(nlayers=n_layers,dropout=0)

In [8]:
print("For the whole process (ie including make_B,B_to_D,D_to_S,cut_S)")
print("Classic : ",end="")
start_time = time.time()
for _ in range(n_test):
    list_kg = pgb.make_all_graphs(
        mod,d,check_device_is_gpu=False,
        bool_kg=False).K_graph_list
    print(".",end="")
print(f"\n-> mean time to generate list_kg: {(time.time() - start_time)/n_test}")

print("Economic : ",end="")
start_time = time.time()
for _ in range(n_test):
    list_sg = pgb.make_all_graphs(
        mod,d,check_device_is_gpu=False,
        bool_kg=False,bool_list_kg=False).S_graph_list
    list_kg_eco = pgb.graph_translator.S_list_to_K_list_eco(list_sg,mod)
    print(".",end="")
print(f"\n-> mean time to generate list_kg_eco: {(time.time() - start_time)/n_test}")

print("="*10)

print("Just to process S_list_to_K_list : ")
print("Classic : ",end="")
start_time = time.time()
for _ in range(n_test):
    list_kg = pgb.Ktools.S_list_to_K_list(list_sg,mod)
    print(".",end="")
print(f"\n-> mean time for S_list_to_K_list: {(time.time() - start_time)/n_test}")

print("Economic : ",end="")
start_time = time.time()
for _ in range(n_test):
    list_kg_eco = pgb.graph_translator.S_list_to_K_list_eco(list_sg,mod)
    print(".",end="")
print(f"\n-> mean time for S_list_to_K_list_eco: {(time.time() - start_time)/n_test}")

For the whole process (ie including make_B,B_to_D,D_to_S,cut_S)
Classic : ...
-> mean time to generate list_kg: 1.8976301352183025
Economic : ...
-> mean time to generate list_kg_eco: 1.4367828369140625
Just to process S_list_to_K_list : 
Classic : ...
-> mean time for S_list_to_K_list: 1.4246115684509277
Economic : ...
-> mean time for S_list_to_K_list_eco: 1.0807079474131267
